In [1]:
from pprint import pprint

import numpy as np
import regex as re
from tiktoken._educational import *

# Let's experiment with gpt2 tokenizer
tokenizer = SimpleBytePairEncoding.from_tiktoken("gpt2")

# `rm -rf tokenization` ?

## Text Tokenization 101

It's a preprocessing step separated from the actual LLM.

It translates human-friendly format (text) to machine-friendly format (numbers).

At its core, the tokenizer is an object with:

In [2]:
print(tokenizer.encode.__doc__)

Encodes a string into tokens.

        >>> enc.encode("hello world")
        [388, 372]
        


In [3]:
print(tokenizer.decode.__doc__)

Decodes a list of tokens into a string.

        Decoded bytes are not guaranteed to be valid UTF-8. In that case, we replace
        the invalid bytes with the replacement character "�".

        >>> enc.decode([388, 372])
        'hello world'
        


In [4]:
print("Vocabolary size:", len(tokenizer.mergeable_ranks))

Vocabolary size: 50256


In LLM, the `list[int]` returned by `tokenizer.encode` are the **row indices of Embedding Matrix `wte_in`** (weights for input text embedding).

In [5]:
                                   # T (time) current context length
V = len(tokenizer.mergeable_ranks) # V vocabolary size
C = 1024                           # C (channel) embedding of size

text_in = "Hello World 👋"
toks_in = tokenizer.encode(text_in, None)  # (T,)
print(f"{text_in=}\n{toks_in=}")

wte_in = np.random.random((V, C)) # [trained]
emb_in = wte_in[toks_in]          # (T, C)

# emb_in -> transformer blocks [trained] -> toks_out
print("\n... LLM computations ...\n")

toks_out = [17250, 30325, 232] # results from LLM
text_out = tokenizer.decode(toks_out)
print(f"{toks_out=}\n{text_out=}")

text_in='Hello World 👋'
toks_in=[15496, 2159, 50169, 233]

... LLM computations ...

toks_out=[17250, 30325, 232]
text_out='Hi 😊'


#### LLM Inference 101

```python
                                       # T (time) current context length
V = len(tokenizer.mergeable_ranks) + 1 # V vocabolary size
C = 1024                               # C (channel) embedding of size

toks_in = [15496, 2159, 50169, 233]
toks_out = []
toks = toks_in + toks_out 

wte_in = np.random.random((V, C))     # [trained]
wte_out = np.random.random((V, C))    # [trained]

def transofrmers(emb_in: np.array) -> np.array: ... # [trained]

while tok_out != 50256: # not "<|endoftext|>"
    
    emb_in = wte_in[toks_in]           # (T, C)
    emb_out = transofrmers(emb_in)     # (T, C)
    logits = wte_out @ emb_out[[-1]].T # (V, C) @ (C, 1) -> (V, 1)
    tok_out = logits.argmax()          # simple sample
    
    toks.append(tok_out)
    toks_out.append(tok_out)
```

### Strings to Integers (Naive)

- Strings in Python are immutable sequences of **Unicode code points**.
- Unicode points map **a character to an integer**.
- The built-in `ord` function returns the Unicode point of a character (a string of length 1).

In [6]:
[ord(c) for c in "Hello World 👋"] 

# Unicode code points

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 32, 128075]

- Unicode defines itself 3 encodings: **UTF-8**, UTF-16, and UTF-32.
- Encodings define how to convert Unicode text (Python strings) to bytes.

In [7]:
list("Hello World 👋".encode("UTF-8")) 

# Bytes (base 10) of UTF-8 encoded string 

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100, 32, 240, 159, 145, 139]

### Why not Unicode code points?

- The Unicode standard is actively developed (variable vocabulary).
- The number of Unicode code points (~154k) ⇒ V is big.
- This results in a long list of tokens ⇒ T is big.
- A single code point does not carry semantic information.
- Rare code points will correspond to undertrained embeddings.

### Why not Raw Bytes?

- Small and fixed vocabulary ⇒ V = 256.
- This results in a really long list of tokens ⇒ T is very big.
- A single byte does not carry semantic information.
- Not all byte streams correspond to valid Unicode text.

---

However, starting from raw bytes, we can use BPE to:

- cleverly extend the vocabulary at our will ⇒ V = 256 + const.
- drastically reduce encoding length T.
- have tokens carrying semantic information.

### Byte Pair Encoding (BPE)

- **→ compress/encode**: from original source to compressed version
- **← decompress/decode**: from compressed version to original version

---
```md
aaabdaaabac ⇄   ZabdZabac   ⇄    ZYdZYac    ⇄    XdXac     [symbols]

    { }       { "Z": "aa" }   { "Z": "aa",    { "Z": "aa", [merges]
                                "Y": "ab" }     "Y": "ab",
                                                "X": "ZY" }
```
---

- *vocabulary size* = num. of raw symbols + num. of merges.
- *"train"* = construct merges dictionary by running BPE over text corpus.
- *"inference"* = make use of merges dictonary to encode/decode.

### Regex Pattern

Simply converting text to bytes and then applying BPE could produce different tokens for: `dog`, `dog.`, `dog!`, and `dog?`.

Therefore, the first step is to disentangle semantic and syntactic elements.

In [8]:
pattern = re.compile(
    r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| """
    r"""?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
)
re.findall(pattern, "Hello World 👋")

['Hello', ' World', ' 👋']

### Tokenization example

Explore various tokenizers interactively on [tiktokenizer](https://tiktokenizer.vercel.app/?model=gpt2)

In [9]:
tokenizer.encode("Hello World 👋") # regex → ['Hello', ' World', ' 👋']

Hello
Hello
Hello
Hello
Hello

 World
 World
 World
 World
 World
 World

 ����
 ���
 ���
 ��



[15496, 2159, 50169, 233]

### Special Tokens

Special tokens correspond to special strings that change/trigger certain behaviors in LLM text generation.

- `<|endoftext|>`: used since GPT-2 to delimit different texts in LLM pre-training

They are also used in fine-tuned models to enforce roles (e.g., system, user, assistant, tool)

- OpenAI (GPT-4): `<|endoftext|>`, `<|im_start|>`, `<|im_end|>`
- Meta (LLaMA 3): `<|begin_of_text|>`, `<|start_header_id|>`, `<|end_header_id|>`, `<|eot_id|>`
- Mistral (Tekken): `<unk>`, `<s>`, `</s>`, `[INST]`, `[/INST]`, `[AVAILABLE_TOOLS]`, `[/AVAILABLE_TOOLS]`, ...

---

They are added as a single token to the vocabulary. For example, in GPT-2,

vocab. size = 256 (raw bytes) + 50000 (BPE merges) + 1 (special token) = 50257

## Issues with Text Tokenization

Much of LLMs' weirdness originates from the tokenization step:

- Low performance at character-level tasks (spelling, reversal, counting)
- Poor performance on non-English languages
- Inability to perform simple arithmetic
- Abrupt halts when encountering certain characters (e.g., "<|endoftext|>")
- Trailing whitespace inconsistencies
- Undertrained embeddings (e.g., "SolidGoldMagikarp")
- Arbitrary implementation choices (e.g., regex patterns)
- ...

## Byte Latent Transformer (BLT)

[paper](https://arxiv.org/abs/2412.09871)

Replace tokenization with a trainable layers that operates on sequence of raw bytes: group them (patch) and generated the corresponding embeddings.

- BLT (8B) match llama 3 (8B) performance with 50% fewer FLOPs.
- BLT more robust to noisy inputs.
- BLT have enhanced preformance for char. level task.
- BLT scales better (more layers, bigger patches) with same FLOPs budget.

*“tokens” refers to byte-groups drawn from a finite vocabulary determined prior to
training as opposed to “patches” which refer to dynamically grouped sequences without a fixed vocabulary.*

###  2. Patching: from indvidual Bytes to group of Bytes

- **Strided Patching Every K Bytes**
    - not clever compute allocation.
    - the same word being differently.
- **Space Patching**
    - cannot gracefully handle all languages and domains
    - cannot vary the patch size
- **Entropy Patching**

#### Entropy Patching

A small char. level LM is used to predict the distribution of the next char.

![Figure4.png](https://media.githubusercontent.com/media/S1M0N38/tokenization-notes/refs/heads/main/figures/Figure4.png?token=AFJ2AVQZ4QTG7C7G6RD33CLHPV36C)

$$
H(x_i) = \sum_{v \in \mathcal{V}} 
p_e (x_i = v | \boldsymbol{x}_{\lt i})
\log p_e (x_i = v | \boldsymbol{x}_{\lt i})
$$

Split at $H(x_t) > \theta_g$ or  $H(x_t) - H(x_{t - 1}) > \theta_r$

![Figure3.png](https://media.githubusercontent.com/media/S1M0N38/tokenization-notes/refs/heads/main/figures/Figure3.png?token=AFJ2AVQHGJC6NCWM6WTSHDLHPV37Q)

### 3 BLT Architecture

![Figure1.png](https://media.githubusercontent.com/media/S1M0N38/tokenization-notes/refs/heads/main/figures/Figure1.png?token=AFJ2AVRCCIYS4S34QDYRU6THPV4BM)

#### Local Encoder & Decoder

![Figure5.png](https://media.githubusercontent.com/media/S1M0N38/tokenization-notes/refs/heads/main/figures/Figure5.png?token=AFJ2AVXB6JDTS4X4DHODUU3HPV4D2)

*Encoder make use of n-gram Embeddings*

### 9 Limitation and Future Work

- Calculation of scaling law for BLT.
- Scale beyond 8b params.
- Runtime optimization (FLOPs != wall-clock time).
- Learning the patching model end-to-end.